<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/Dynamic_fire_test_urban.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%%time 
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

# install rasterio
!pip install rasterio[3]

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
4 packages can be upgraded. Run 'apt

In [7]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from osgeo import gdal_array
import numpy as np
import pandas as pd
import imageio
import random

In [12]:
path = '/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/Colab_folder'
!ls '/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/Colab_folder'

GD_fuel_map.tif  GD_ignit_prob.tif  GD_wind.csv


In [14]:
def load_data(path_to_data=path):
    # load building map as a "fuel" map
    fuel = gdal_array.LoadFile(os.path.join(path_to_data, 'GD_fuel_map.tif'))  # 0 = no fuel; 1 = fuel
    # load probability of building ignition as an array
    ignition = gdal_array.LoadFile(os.path.join(path_to_data, 'GD_ignit_prob.tif'))  # probability from 0 to 1
    # load wind data
    wind = pd.read_csv(os.path.join(path_to_data, 'GD_wind.csv'))
    return fuel, ignition, wind


fuel_map, ignition_probability_map, wind_df = load_data()

AttributeError: ignored

In [0]:
def wind_scenario(wind_data=wind_df):
    i = np.random.randint(0, wind_data.shape[0])
    wind = wind_data.iloc[i, 2]
    distance = wind_data.iloc[i, 1]
    return wind, distance


wind_direction, critical_distance = wind_scenario()

In [0]:
def fire_propagation(scenarios, fuel=fuel_map, wind=wind_direction, distance=critical_distance,
                     ignition_map=ignition_probability_map):

    for scenario in range(scenarios):
        # fire hold the state of each cell
        time_total = 1000
        ignition = np.zeros((time_total, *fuel))
        fire = np.zeros((time_total, *fuel))
        fire_list = []
        # initialize fire by creating random ignition from ignition probability map
        ignition[0] = np.random.choice([0, 1], size=fuel,
                                       p=ignition_map)  # ignition must not happen in non fuel cells !!
        fire[0] = ignition[0] + fuel  # 0 = no fuel, 1 = fuel, 2 = fire

        for time in range(1, time_total, 1):
            # Make a copy of the original fire
            fire[time] = fire[time - 1].copy()
            for x in range(1, fuel[0] - 1):
                for y in range(1, fuel[1] - 1):
                    for d in range(distance, 1, -1):
                        if fire[time - 1, x, y] == 2 and wind == 'buffer':  # It's on fire
                            # If there's fuel surrounding it
                            # set it on fire!
                            if fire[time - 1, x - distance, y + distance] == 1:
                                fire[time, x - distance, y + distance] = 2
                            if fire[time - 1, x, y + distance] == 1:
                                fire[time, x, y + distance] = 2
                            if fire[time - 1, x + distance, y + distance] == 1:
                                fire[time, x + distance, y + distance] = 2
                            if fire[time - 1, x + distance, y] == 1:
                                fire[time, x + distance, y] = 2
                            if fire[time - 1, x + distance, y - distance] == 1:
                                fire[time, x + distance, y - distance] = 2
                            if fire[time - 1, x, y - distance] == 1:
                                fire[time, x, y - distance] = 2
                            if fire[time - 1, x - distance, y - distance] == 1:
                                fire[time, x - distance, y - distance] = 2
                            if fire[time - 1, x - distance, y] == 1:
                                fire[time, x - distance, y] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'N':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x - distance, y + distance] == 1:
                                fire[time, x - distance, y + distance] = 2
                            if fire[time - 1, x, y + distance] == 1:
                                fire[time, x, y + distance] = 2
                            if fire[time - 1, x + distance, y + distance] == 1:
                                fire[time, x + distance, y + distance] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'NE':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x, y + distance] == 1:
                                fire[time, x, y + distance] = 2
                            if fire[time - 1, x + distance, y + distance] == 1:
                                fire[time, x + distance, y + distance] = 2
                            if fire[time - 1, x + distance, y] == 1:
                                fire[time, x + distance, y] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'E':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x + distance, y + distance] == 1:
                                fire[time, x + distance, y + distance] = 2
                            if fire[time - 1, x + distance, y] == 1:
                                fire[time, x + distance, y] = 2
                            if fire[time - 1, x + distance, y - distance] == 1:
                                fire[time, x + distance, y + distance] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'SE':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x + distance, y] == 1:
                                fire[time, x + distance, y] = 2
                            if fire[time - 1, x + distance, y - distance] == 1:
                                fire[time, x + distance, y + distance] = 2
                            if fire[time - 1, x, y - distance] == 1:
                                fire[time, x, y - distance] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'S':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x, y - distance] == 1:
                                fire[time, x, y - distance] = 2
                            if fire[time - 1, x - distance, y - distance] == 1:
                                fire[time, x - distance, y - distance] = 2
                            if fire[time - 1, x + distance, y - distance] == 1:
                                fire[time, x + distance, y - distance] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'SW':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x, y - distance] == 1:
                                fire[time, x, y - distance] = 2
                            if fire[time - 1, x - distance, y - distance] == 1:
                                fire[time, x - distance, y - distance] = 2
                            if fire[time - 1, x - distance, y] == 1:
                                fire[time, x - distance, y] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'W':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x - distance, y + distance] == 1:
                                fire[time, x - distance, y + distance] = 2
                            if fire[time - 1, x - distance, y - distance] == 1:
                                fire[time, x - distance, y - distance] = 2
                            if fire[time - 1, x - distance, y] == 1:
                                fire[time, x - distance, y] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'NW':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x - distance, y + distance] == 1:
                                fire[time, x - distance, y + distance] = 2
                            if fire[time - 1, x - distance, y] == 1:
                                fire[time, x - distance, y] = 2
                            if fire[time - 1, x, y + distance] == 1:
                                fire[time, x, y + distance] = 2

                          fire[time].plot()

                        if np.array_equal(fire[time], fire[time - 1]) == True:
                            fire_list.append(fire[time])
                            pass
                        else:
                            continue
        return fire_list


final_fire_maps = fire_propagation(1)
